In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [12]:

# def make_block(block, num_residual_blocks, intermediate_channels, stride ):
#     identity_downsample = None


#     layers = [
#         nn.Sequential(
#             nn.Conv2d(
#                 in_channels, out_channels,
#                 kernel_size=kernel_size, stride=stride,
#                 padding=padding,
#                 bias=bias
#             ),
#             nn.ReLU(inplace=True)
#         ) for in_channels, out_channels, stride in zip(channels[:-1], channels[1:], stride_block[:])
#     ]
    
#     return nn.Sequential(*layers)


# def _make_layer(block, in_channels, intermediate_channels, stride ):


class layer(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1 ,identity_downsample=None):
        super().__init__()
        self.identity_downsample = identity_downsample
        self.expansion = 4
        self.conv_1 = nn.Conv2d(in_channels, out_channels, kernel_size=1,stride=1, padding=0)
        self.conv_2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.conv_3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
    
    def forward(self, x):
        identity = x.clone()

        out = self.conv_1(x)
        out = self.conv_2(out)
        out = self.conv_3(out)

        if self.identity_downsample:
            identity = self.identity_downsample(x)

        out = out + identity

        return out


        


class Resnet(nn.Module):
    
    def __init__(
            self,
            block_out = [64, 64, 128, 256, 512 ],
            block_depth = [6,8,12,5],
            kernel_size_block = [3, 3],
            stride_block = [1, 2, 2, 2],
            padding_block = [1,1],
            
    ):
        super().__init__()
        
        self.in_channels = 64
        blocks = self.make_block(layer,3,intermediate_channels=64, stride=1)
        
        self.model = nn.Sequential(
            nn.Conv2d(3,64, 7,2,3, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3,2,padding=1),
            # *blocks,
            # nn.Conv2d(64, 64, 3, 1, 1, bias=False),
            # nn.Conv2d(64, 128, 3, 2, 1, bias= False),
        )
        
    
    def forward(self, x):
        out = self.model(x)
        # out = self.conv_2(out)
        # out = self.pool_1(out)
        return out 
    
    def make_block(self, make_layer, num_residual_blocks, intermediate_channels, stride):

        identity_downsample = None 
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(
                self.in_channels,
                intermediate_channels*4,
                kernel_size=1,
                stride=stride,
                bias=False
            ))


        layers.append(make_layer(self.in_channels, intermediate_channels, stride, identity_downsample))

        self.in_channels = intermediate_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(make_layer(self.in_channels, intermediate_channels))

        
        return nn.Sequential(*layers)

def test_resnet():
    model = Resnet()
    x = torch.randn(4, 3, 224, 224)
    out = model(x)
    print(out.shape)
    print(model)

test_resnet()

torch.Size([4, 64, 56, 56])
Resnet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
)
